In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
%cd drive/MyDrive/Classroom/

In [ ]:
import tarfile


fname = 'fer2013.tar.gz'
if fname.endswith("tar.gz"):
    tar = tarfile.open(fname, "r:gz")
    tar.extractall()
    tar.close()
elif fname.endswith("tar"):
    tar = tarfile.open(fname, "r:")
    tar.extractall()
    tar.close()

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv('fer2013/fer2013.csv')
df.head()

In [ ]:
df.emotion.unique()

In [ ]:
label_to_text = {0:'Abdullah', 1:'Elon', 2:'Jeff', 3:'Bill'} 

In [ ]:
np.array(df.pixels.loc[0].split(' ')).reshape(48,48)

In [ ]:
pyplot.imshow(np.array(df.pixels.loc[0].split(' ')).reshape(48,48).astype('float'))

In [ ]:
fig = pyplot.figure(1, (14, 14))
k = 0
for label in sorted(df.emotion.unique()):
    for j in range(3):
        px = df[df.emotion==label].pixels.iloc[k]
        px = np.array(px.split(' ')).reshape(48, 48).astype('float32')
        k += 1
        ax = pyplot.subplot(7, 7, k)
        ax.imshow(px)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(label_to_text[label])
        pyplot.tight_layout()
  

In [ ]:
img_array= df.pixels.apply(lambda x: np.array(x.split(' ')).reshape(48, 48).astype('float32'))

In [ ]:
img_array = np.stack(img_array,axis=0)

In [ ]:
img_array.shape

In [ ]:
lables = df.emotion.values

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(img_array,lables,test_size=.1)

In [ ]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

In [ ]:
x_train=x_train/255
x_test=x_test/255


In [ ]:
basemodel=tf.keras.models.Sequential([tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape =(48,48,1))
                                    ,tf.keras.layers.MaxPool2D(2,2),
                                    tf.keras.layers.BatchNormalization(),
                                    
                                    tf.keras.layers.Conv2D(64,(3,3),activation='relu',input_shape =(48,48,1))
                                    ,tf.keras.layers.MaxPool2D(2,2),
                                    tf.keras.layers.BatchNormalization(),


                                    tf.keras.layers.Conv2D(128,(3,3),activation='relu',input_shape =(48,48,1))
                                    ,tf.keras.layers.MaxPool2D(2,2),
                                    tf.keras.layers.BatchNormalization(),
                                    

                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128,activation='relu'),
                                    tf.keras.layers.Dense(7,activation='softmax')
                                    
                                    ])

In [ ]:
basemodel.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
import os
try:
  os.mkdir('checkpoint')
except:
  pass

In [ ]:
file_name ='best_model.h5'
checkpoint_path=os.path.join('checkpoint',file_name)

call_back=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                             monitor='val_accuracy',
                                             verbose=1,
                                             save_freq="epoch",
                                             save_best_only=True,
                                             save_weights_only=False,
                                             mode='max')

In [ ]:
basemodel.fit(x_train,y_train,epochs=20,validation_split=0.1,callbacks=call_back)